# Generar entorno virtual
1. Correr desde CMD dentro de VS Code
2. crear el venv: `python -m venv .venv`
3. activarlo: `.\.venv\Scripts\activate.bat`

# Librerias, imports, carpetas

In [1]:
# Instalando librerias por 1era vez
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine
%pip install pandas
%pip install numpy
%pip install seaborn
%pip install matplotlib
%pip install datetime
%pip install scikit-learn
%pip install plotly

#Si hay algun error lo solucionamos instalando librerias especificas
#%pip install scikit-learn==1.3.2
#%pip install seaborn==0.13.1
#%pip install numpy==1.26.4
#%pip install matplotlib==3.7.1
#%pip install optuna==3.6.1

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# #Importando librerias al inicio de todo script.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import plotly

In [3]:
#Configurando duckDB
import duckdb
# Carga la extensión de SQL (solo funciona en notebooks)
%load_ext sql

# Configura las opciones para que SQL devuelva DataFrames
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Conecta a una base de datos DuckDB en memoria (temporal)
%sql duckdb://

In [4]:
#Generando directorios y paths
#Paths
PATH_BASE = "C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01" #Sacar la ruta con icono de carpeta del indice/outline si trabajo en G.Drive.

#Nombres de carpetas y archivos
folders = ["models", "kaggle_submissions", "optimization_trees_optuna"]

# Crear carpetas
for f in folders:
    path = os.path.join(PATH_BASE, f)
    os.makedirs(path, exist_ok=True)
    print(f"✅ Carpeta creada / ya existente: {path}")
    
#Nombres de archivos
FILENAME_CRUDO = 'competencia_01_crudo.csv'
FILENAME_CLASE_TERNARIA = 'competencia_01_clase_ternaria.csv'

#Paths
# Diccionario para guardar rutas completas
paths = {}

# Crear carpetas y guardar rutas
for folder in folders:
    full_path = os.path.join(PATH_BASE, folder)
    os.makedirs(full_path, exist_ok=True)
    print(f"✅ Carpeta lista: {full_path}")
    paths[folder.upper()] = full_path

# Crear variables individuales (opcional)
PATH_MODELS = paths["MODELS"]
PATH_KAGGLE_SUBMISSIONS = paths["KAGGLE_SUBMISSIONS"]
PATH_OPTIMIZATION_TREES_OPTUNA = paths["OPTIMIZATION_TREES_OPTUNA"]
PATH_DATA = "/".join([PATH_BASE, "data"])


✅ Carpeta creada / ya existente: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01\models
✅ Carpeta creada / ya existente: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01\kaggle_submissions
✅ Carpeta creada / ya existente: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01\optimization_trees_optuna
✅ Carpeta lista: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01\models
✅ Carpeta lista: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01\kaggle_submissions
✅ Carpeta lista: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01\optimization_trees_optuna


# Clase ternaria

El dataset crudo viene sin la columna target a predecir, hay que generarla.

In [5]:
#Levando la data
PATH_CRUDO ="/".join([PATH_DATA, FILENAME_CRUDO])

Para cargar el archivo .csv a una tabla pasando su ubicación a través de la variable `PATH_CRUDO`

In [6]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{PATH_CRUDO}}")

,Success


In [7]:
df_original = pd.read_csv(os.path.join(PATH_CRUDO))  # ---> Fuente datos: https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv

In [8]:
print(f"✅ Data cargada con filas: {df_original.shape[0]} y columnas: {df_original.shape[1]}")

✅ Data cargada con filas: 978439 y columnas: 154


**Creando la target: Clase Ternaria**

Clase ternaria:

* BAJA+1 → está en m, pero no en m+1.
* BAJA+2 → está en m, está en m+1, pero no en m+2.
* CONTINUA → está en m, m+1 y m+2.

In [16]:
%%sql
create or replace table base_target as (
select numero_de_cliente,
       foto_mes,
       case when foto_mes_1 is null then 'BAJA+1'
            when foto_mes_2 is null then 'BAJA+2'
            else 'CONTINUA'
       end as clase_ternaria
from (
select numero_de_cliente,
       foto_mes,
       LEAD(foto_mes, 1, NULL) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) foto_mes_1,
       LEAD(foto_mes, 2, NULL) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) foto_mes_2
from competencia_01_crudo) as a
where foto_mes <= 202104)

,Success


Con esta linea me guardo en un data frame de python: el nro de cliente, foto_mes y la target.

In [17]:
# Usar el result set como pandas DataFrame
df_target = %sql SELECT * FROM base_target;

In [18]:
# Hacemos el merge con df_original
df_merged = pd.merge(
    df_original,              # tu DataFrame original
    df_target,       # el que contiene la variable target
    on=["numero_de_cliente", "foto_mes"],  # claves comunes
    how="left"       # conservamos todas las filas del original
)

Finalmente guardo el dataset para ser usado posteriormente

In [21]:
output_path = "/".join([PATH_DATA, FILENAME_CLASE_TERNARIA])
df_merged.to_csv(output_path, index=False)
# Confirmar ruta
print(f"✅ Archivo guardado en: {output_path}")

✅ Archivo guardado en: C:/Users/ybbar/OneDrive/Desktop/DMEyF2025_Competencia01/data/competencia_01_clase_ternaria.csv
